###### December 2019 - Roger Melko

Diagonalizing the Hamiltonian matrix for the transverse field Ising model to find the energy eigenvalues and eigenkets.   Calculate the groundstate magnetization.

We will use the same Hamiltonian convention as the QMC program:
$$
H = -J\sum_{\langle i j \rangle} \sigma^z_i \sigma^z_j - B \sum_i \sigma^x_i
$$
where ${\bf \sigma}_i$ are Pauli operators.  In this convention, the 1+1 CFT is at $h/J = 1$.

In [12]:
N = 10
Dim = 2^N

J = 1. #this is the exchange
B = 1. #this is the transverse field

Hamiltonian = zeros(Dim,Dim)   #This is your 2D Hamiltonian matrix

for Ket = 0:Dim-1  #Loop over Hilbert Space
    Diagonal = 0.
    for SpinIndex = 0:N-2  #Loop over spin index (base zero, stop one spin before the end of the chain)
        Spin1 = 2*((Ket>>SpinIndex)&1) - 1
        NextIndex = SpinIndex + 1
        Spin2 = 2*((Ket>>NextIndex)&1) - 1
        Diagonal = Diagonal - J*Spin1*Spin2 #spins are +1 and -1
    end
    Hamiltonian[Ket+1,Ket+1] = Diagonal
    
    for SpinIndex = 0:N-1
        bit = 2^SpinIndex   #The "label" of the bit to be flipped
        Bra = Ket ⊻ bit    #Binary XOR flips the bit
        Hamiltonian[Bra+1,Ket+1] = -B
    end
    
end

In the Julia LinearAlgebra package, the eigen function finds eigenvalues and eigenvectors.  They are ordered; i.e. the groundstate energy corresponds to index 1

In [15]:
using LinearAlgebra
Diag = eigen(Hamiltonian)     #Diagonalize the Hamiltonian
# println(Diag.values)         #This is all of the eigenvalues
#index = findmin(Diag.values)  #The minimum eigenvalue corresponds to index=1
#println(index[1])
GroundState = Diag.vectors[:, 1];  #this gives the groundstate eigenvector
# println(GroundState)

In [16]:
Diag.values[1] / N

-1.2381489999654751

In [17]:
##### Calculate the groundstate magnetization <m^2> in the Z direction
magnetization = 0
for Ket = 0:Dim-1  #Loop over Hilbert Space
    SumSz = 0.
    for SpinIndex = 0:N-1  #Loop over spin index (base zero, stop one spin before the end of the chain)
        Spin1 = 2*((Ket>>SpinIndex)&1) - 1
        SumSz += Spin1 #spin is +1 or -1
        #print(Spin1," ")
    end
    #println(SumSz," ",GroundState[Ket+1])
    magnetization += SumSz*SumSz*GroundState[Ket+1]^2  #Don't forgot to square the coefficients...
end
println(magnetization/(N*N))    

0.41153322728818764
